## import Libraries and Prepare Data

In [13]:
# Import Libraries and Prepare Data
import warnings

# Suppress warnings
warnings.filterwarnings("ignore", category=UserWarning)

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from PIL import Image
import matplotlib.pyplot as plt
import os
import copy

In [14]:
# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [15]:
# Image size
image_size = 512 if torch.cuda.is_available() else 128
print(f"Image size: {image_size}")

Image size: 128


In [16]:
# Define transform
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
])

In [17]:
# Load image function
def load_image(image_path, size=None):
    image = Image.open(image_path)
    if size is not None:
        image = transforms.Resize(size)(image)
    image = transform(image).unsqueeze(0)
    return image.to(device)

# Get image size function
def get_image_size(image):
    return image.shape[2], image.shape[3]

# Imshow function
def imshow(tensor, title=None):
    image = tensor.cpu().clone().detach().squeeze(0)
    image = transforms.ToPILImage()(image)

    # Resize the image to match the smallest dimension (width or height)
    min_size = min(image.size)
    transform_resize = transforms.Resize(min_size)
    image = transform_resize(image)

    plt.imshow(image)
    if title is not None:
        plt.title(title)
    plt.show()

In [18]:

# Define StyleTransfer class
class StyleTransfer(nn.Module):
    def __init__(self, cnn, normalization_mean, normalization_std, content_image, style_image, content_layers=None, style_layers=None):
        super(StyleTransfer, self).__init__()
        self.cnn = copy.deepcopy(cnn)
        self.normalization = Normalization(normalization_mean, normalization_std).to(device)

        if content_layers is None:
            content_layers = ['conv_4']
        if style_layers is None:
            style_layers = ['conv_1', 'conv_2', 'conv_3', 'conv_4', 'conv_5']

        self.content_layers = content_layers
        self.style_layers = style_layers

        self.model, self.content_losses, self.style_losses = self.get_model_and_losses(content_image, style_image)

    def get_model_and_losses(self, content_image, style_image):
        content_losses = []
        style_losses = []
        model = nn.Sequential(self.normalization)
        i = 0

        for layer in self.cnn.children():
            if isinstance(layer, nn.Conv2d):
                i += 1
                name = f'conv_{i}'
            elif isinstance(layer, nn.ReLU):
                name = f'relu_{i}'
                layer = nn.ReLU(inplace=False)
            elif isinstance(layer, nn.MaxPool2d):
                name = f'pool_{i}'
            elif isinstance(layer, nn.BatchNorm2d):
                name = f'bn_{i}'
            else:
                raise RuntimeError(f'Unrecognized layer: {layer.__class__.__name__}')

            model.add_module(name, layer)

            if name in self.content_layers:
                target = model(content_image).detach()
                content_loss = ContentLoss(target)
                model.add_module(f"content_loss_{i}", content_loss)
                content_losses.append(content_loss)

            if name in self.style_layers:
                target_feature = model(style_image).detach()
                style_loss = StyleLoss(target_feature)
                model.add_module(f"style_loss_{i}", style_loss)
                style_losses.append(style_loss)

        for i in range(len(model) - 1, -1, -1):
            if isinstance(model[i], ContentLoss) or isinstance(model[i], StyleLoss):
                break

        model = model[:(i + 1)]

        return model, content_losses, style_losses

    def gram_matrix(self, input):
        a, b, c, d = input.size()
        features = input.view(a * b, c * d)
        G = torch.mm(features, features.t())
        return G.div(a * b * c * d)

# Define Normalization class
class Normalization(nn.Module):
    def __init__(self, mean, std):
        super(Normalization, self).__init__()
        self.mean = torch.tensor(mean).view(-1, 1, 1).to(device)
        self.std = torch.tensor(std).view(-1, 1, 1).to(device)

    def forward(self, img):
        return (img - self.mean) / self.std

# Define ContentLoss class
class ContentLoss(nn.Module):
    def __init__(self, target):
        super(ContentLoss, self).__init__()
        self.target = target.detach()

    def forward(self, input):
        self.loss = nn.functional.mse_loss(input, self.target)
        return input

# Define StyleLoss class
class StyleLoss(nn.Module):
    def __init__(self, target_feature):
        super(StyleLoss, self).__init__()
        self.target = StyleTransfer.gram_matrix(self, target_feature).detach()

    def forward(self, input):
        G = StyleTransfer.gram_matrix(self, input)
        self.loss = nn.functional.mse_loss(G, self.target)
        return input

In [19]:
# Get input optimizer function
def get_input_optimizer(input_img):
    optimizer = optim.LBFGS([input_img.requires_grad_()])
    return optimizer

# Run style transfer function
def run_style_transfer(style_transfer, input_img, num_steps=500, style_weight=1000000, content_weight=1):
    optimizer = get_input_optimizer(input_img)
    run = [0]
    while run[0] <= num_steps:

        def closure():
            input_img.data.clamp_(0, 1)
            optimizer.zero_grad()
            style_transfer.model(input_img)
            style_score = 0
            content_score = 0

            for sl in style_transfer.style_losses:
                style_score += sl.loss
            for cl in style_transfer.content_losses:
                content_score += cl.loss

            style_score *= style_weight
            content_score *= content_weight

            loss = style_score + content_score
            loss.backward()

            run[0] += 1
            if run[0] % 10 == 0:
                print(f"run {run[0]}:")
                print(f'Style Loss : {style_score.item():4f} Content Loss: {content_score.item():4f}')
                print()

            return loss

        optimizer.step(closure)
    input_img.data.clamp_(0, 1)
    return input_img

In [20]:
# Load pre-trained VGG19 model
cnn = models.vgg19(pretrained=True).features.to(device).eval()

In [21]:
# Normalization mean and std
cnn_normalization_mean = torch.tensor([0.485, 0.456, 0.406]).to(device)
cnn_normalization_std = torch.tensor([0.229, 0.224, 0.225]).to(device)

In [22]:
# Define directories
working_dir = os.getcwd()
content_dir = os.path.join(working_dir, "content")
style_dir = os.path.join(working_dir, "style")
output_dir = os.path.join(working_dir, "output")
os.makedirs(output_dir, exist_ok=True)

In [24]:
# Load content and style images
content_images = os.listdir(content_dir)
style_images = os.listdir(style_dir)

# Perform style transfer for each pair of content and style images
for content_image_name in content_images:
    for style_image_name in style_images:
        content_image_path = os.path.join(content_dir, content_image_name)
        style_image_path = os.path.join(style_dir, style_image_name)
        output_image_name = f"{os.path.splitext(content_image_name)[0]}_{os.path.splitext(style_image_name)[0]}.jpg"
        print("Building model for content image", os.path.splitext(content_image_name)[0], "Vs style image", os.path.splitext(style_image_name)[0])
        content_image = load_image(content_image_path)
        style_image = load_image(style_image_path, get_image_size(content_image))

        style_transfer = StyleTransfer(cnn, cnn_normalization_mean, cnn_normalization_std, content_image, style_image)

        input_image = content_image.clone()

        output_image = run_style_transfer(style_transfer, input_image)

        output_image_path = os.path.join(output_dir, output_image_name)
        output_image_pil = transforms.ToPILImage()(output_image.squeeze(0).cpu())
        output_image_pil.save(output_image_path)

        print(f"Saved stylized image: {output_image_path}")



Building model for content image content1 Vs style image art1
run 10:
Style Loss : 127855.625000 Content Loss: 135.313660

run 20:
Style Loss : 8924.048828 Content Loss: 126.005173

run 30:
Style Loss : 2987.827148 Content Loss: 131.940308

run 40:
Style Loss : 1615.333984 Content Loss: 130.763153

run 50:
Style Loss : 1026.220825 Content Loss: 130.999268

run 60:
Style Loss : 674.962402 Content Loss: 130.744324

run 70:
Style Loss : 535.509216 Content Loss: 130.902725

run 80:
Style Loss : 422.898895 Content Loss: 130.735962

run 90:
Style Loss : 367.287476 Content Loss: 130.411606

run 100:
Style Loss : 306.527649 Content Loss: 130.396698

run 110:
Style Loss : 265.237000 Content Loss: 130.166656

run 120:
Style Loss : 225.152298 Content Loss: 129.950394

run 130:
Style Loss : 194.909805 Content Loss: 129.840576

run 140:
Style Loss : 178.593857 Content Loss: 129.623978

run 150:
Style Loss : 284.529602 Content Loss: 128.073730

run 160:
Style Loss : 2368.310791 Content Loss: 127.846

run 320:
Style Loss : 3.251623 Content Loss: 25.361769

run 330:
Style Loss : 3.190680 Content Loss: 25.107681

run 340:
Style Loss : 3.073864 Content Loss: 24.880346

run 350:
Style Loss : 3.016121 Content Loss: 24.678240

run 360:
Style Loss : 2.934073 Content Loss: 24.470181

run 370:
Style Loss : 2.877043 Content Loss: 24.293333

run 380:
Style Loss : 2.821022 Content Loss: 24.108139

run 390:
Style Loss : 2.753319 Content Loss: 23.940952

run 400:
Style Loss : 2.707409 Content Loss: 23.777491

run 410:
Style Loss : 2.667620 Content Loss: 23.598969

run 420:
Style Loss : 2.635102 Content Loss: 23.450684

run 430:
Style Loss : 2.586817 Content Loss: 23.317661

run 440:
Style Loss : 2.553281 Content Loss: 23.185928

run 450:
Style Loss : 2.500376 Content Loss: 23.069483

run 460:
Style Loss : 2.476557 Content Loss: 22.943363

run 470:
Style Loss : 2.452163 Content Loss: 22.802267

run 480:
Style Loss : 2.432235 Content Loss: 22.690205

run 490:
Style Loss : 2.416106 Content Loss: 22.

run 100:
Style Loss : 337.534851 Content Loss: 130.011353

run 110:
Style Loss : 280.552307 Content Loss: 129.800354

run 120:
Style Loss : 222.815353 Content Loss: 129.730713

run 130:
Style Loss : 183.202606 Content Loss: 129.578476

run 140:
Style Loss : 151.177521 Content Loss: 129.212204

run 150:
Style Loss : 127.767471 Content Loss: 128.936005

run 160:
Style Loss : 110.947891 Content Loss: 128.665680

run 170:
Style Loss : 93.405670 Content Loss: 128.255402

run 180:
Style Loss : 81.843269 Content Loss: 128.036987

run 190:
Style Loss : 74.788742 Content Loss: 127.822975

run 200:
Style Loss : 68.395065 Content Loss: 127.614899

run 210:
Style Loss : 63.416710 Content Loss: 127.322395

run 220:
Style Loss : 57.400578 Content Loss: 127.146126

run 230:
Style Loss : 52.850254 Content Loss: 126.911461

run 240:
Style Loss : 49.187359 Content Loss: 126.669151

run 250:
Style Loss : 47.528030 Content Loss: 126.275307

run 260:
Style Loss : 44.459988 Content Loss: 125.965622

run 270

run 440:
Style Loss : 2.839921 Content Loss: 18.290785

run 450:
Style Loss : 2.819792 Content Loss: 18.188461

run 460:
Style Loss : 2.678123 Content Loss: 18.134460

run 470:
Style Loss : 2.624466 Content Loss: 18.056049

run 480:
Style Loss : 2.594150 Content Loss: 17.975603

run 490:
Style Loss : 2.567934 Content Loss: 17.883112

run 500:
Style Loss : 2.539062 Content Loss: 17.803509

run 510:
Style Loss : 2.515950 Content Loss: 17.732502

run 520:
Style Loss : 2.503479 Content Loss: 17.657391

Saved stylized image: C:\Users\jains\Downloads\Neural-Style-Transfer-main\Neural-Style-Transfer-main\output\content2_art3.jpg
Building model for content image content2 Vs style image art4
run 10:
Style Loss : 5165.522949 Content Loss: 23.068970

run 20:
Style Loss : 612.081299 Content Loss: 26.701874

run 30:
Style Loss : 290.651398 Content Loss: 27.894505

run 40:
Style Loss : 186.699127 Content Loss: 28.158596

run 50:
Style Loss : 125.761772 Content Loss: 28.759403

run 60:
Style Loss : 8

run 220:
Style Loss : 62.179417 Content Loss: 120.099228

run 230:
Style Loss : 52.661140 Content Loss: 120.163635

run 240:
Style Loss : 49.062077 Content Loss: 120.046219

run 250:
Style Loss : 46.167225 Content Loss: 119.924286

run 260:
Style Loss : 43.571827 Content Loss: 119.708481

run 270:
Style Loss : 39.864563 Content Loss: 119.610275

run 280:
Style Loss : 37.769943 Content Loss: 119.420380

run 290:
Style Loss : 35.345993 Content Loss: 119.194550

run 300:
Style Loss : 33.753773 Content Loss: 118.986771

run 310:
Style Loss : 32.410885 Content Loss: 118.907532

run 320:
Style Loss : 31.876978 Content Loss: 118.622772

run 330:
Style Loss : 29.382572 Content Loss: 118.614975

run 340:
Style Loss : 28.206734 Content Loss: 118.444176

run 350:
Style Loss : 660.060120 Content Loss: 115.850685

run 360:
Style Loss : 52.244465 Content Loss: 118.165337

run 370:
Style Loss : 32.458569 Content Loss: 118.145050

run 380:
Style Loss : 27.974318 Content Loss: 118.083405

run 390:
Styl

run 10:
Style Loss : 10259.740234 Content Loss: 30.563660

run 20:
Style Loss : 1230.407715 Content Loss: 34.907230

run 30:
Style Loss : 602.384827 Content Loss: 34.748390

run 40:
Style Loss : 338.984192 Content Loss: 34.553146

run 50:
Style Loss : 238.830597 Content Loss: 34.458702

run 60:
Style Loss : 180.471405 Content Loss: 34.507500

run 70:
Style Loss : 140.807251 Content Loss: 34.388977

run 80:
Style Loss : 114.108986 Content Loss: 34.416393

run 90:
Style Loss : 96.524734 Content Loss: 34.486214

run 100:
Style Loss : 83.360847 Content Loss: 34.583687

run 110:
Style Loss : 71.373375 Content Loss: 34.590733

run 120:
Style Loss : 61.990723 Content Loss: 34.572578

run 130:
Style Loss : 54.857712 Content Loss: 34.589222

run 140:
Style Loss : 47.976910 Content Loss: 34.566212

run 150:
Style Loss : 42.566513 Content Loss: 34.560661

run 160:
Style Loss : 38.111031 Content Loss: 34.524654

run 170:
Style Loss : 33.699108 Content Loss: 34.466873

run 180:
Style Loss : 30.4093

run 340:
Style Loss : 29.188429 Content Loss: 128.099976

run 350:
Style Loss : 27.818142 Content Loss: 127.832642

run 360:
Style Loss : 27.523365 Content Loss: 127.408531

run 370:
Style Loss : 33.372185 Content Loss: 126.804260

run 380:
Style Loss : 25.753031 Content Loss: 126.835541

run 390:
Style Loss : 27.651516 Content Loss: 126.174942

run 400:
Style Loss : 24.547401 Content Loss: 126.099922

run 410:
Style Loss : 24.410398 Content Loss: 125.599274

run 420:
Style Loss : 23.631769 Content Loss: 125.211151

run 430:
Style Loss : 24.871878 Content Loss: 125.003258

run 440:
Style Loss : 22.532032 Content Loss: 124.753273

run 450:
Style Loss : 22.287745 Content Loss: 124.196976

run 460:
Style Loss : 23.006899 Content Loss: 123.799049

run 470:
Style Loss : 21.822201 Content Loss: 123.614906

run 480:
Style Loss : 21.310877 Content Loss: 123.213043

run 490:
Style Loss : 25.929464 Content Loss: 122.647408

run 500:
Style Loss : 20.302647 Content Loss: 122.586182

run 510:
Style

run 140:
Style Loss : 17.477665 Content Loss: 21.268829

run 150:
Style Loss : 16.014061 Content Loss: 21.083391

run 160:
Style Loss : 14.697957 Content Loss: 20.922443

run 170:
Style Loss : 13.497620 Content Loss: 20.746803

run 180:
Style Loss : 12.528601 Content Loss: 20.586405

run 190:
Style Loss : 11.638713 Content Loss: 20.429586

run 200:
Style Loss : 10.830431 Content Loss: 20.248451

run 210:
Style Loss : 10.093414 Content Loss: 20.091497

run 220:
Style Loss : 9.515430 Content Loss: 19.944611

run 230:
Style Loss : 8.925425 Content Loss: 19.809706

run 240:
Style Loss : 8.431258 Content Loss: 19.682343

run 250:
Style Loss : 7.964051 Content Loss: 19.576799

run 260:
Style Loss : 7.548684 Content Loss: 19.444925

run 270:
Style Loss : 7.182364 Content Loss: 19.319845

run 280:
Style Loss : 6.755840 Content Loss: 19.209835

run 290:
Style Loss : 6.436816 Content Loss: 19.114891

run 300:
Style Loss : 6.124323 Content Loss: 19.015997

run 310:
Style Loss : 5.855070 Content L

run 470:
Style Loss : 20.982382 Content Loss: 120.585838

run 480:
Style Loss : 20.145918 Content Loss: 120.294479

run 490:
Style Loss : 19.723663 Content Loss: 120.074776

run 500:
Style Loss : 19.552444 Content Loss: 119.757492

run 510:
Style Loss : 19.390312 Content Loss: 119.457199

run 520:
Style Loss : 18.874603 Content Loss: 119.364326

Saved stylized image: C:\Users\jains\Downloads\Neural-Style-Transfer-main\Neural-Style-Transfer-main\output\neural_style_transfer_5_0_art1.jpg
Building model for content image neural_style_transfer_5_0 Vs style image art2
run 10:
Style Loss : 2185523.750000 Content Loss: 192.629608

run 20:
Style Loss : 1866.192993 Content Loss: 53.080070

run 30:
Style Loss : 625.987122 Content Loss: 53.523258

run 40:
Style Loss : 276.535339 Content Loss: 53.813477

run 50:
Style Loss : 148.606110 Content Loss: 53.390957

run 60:
Style Loss : 86.794510 Content Loss: 52.723934

run 70:
Style Loss : 61.160107 Content Loss: 52.485157

run 80:
Style Loss : 44.550

run 250:
Style Loss : 13.558992 Content Loss: 23.600586

run 260:
Style Loss : 12.538262 Content Loss: 23.479191

run 270:
Style Loss : 11.782739 Content Loss: 23.375923

run 280:
Style Loss : 11.156839 Content Loss: 23.257296

run 290:
Style Loss : 10.525211 Content Loss: 23.124250

run 300:
Style Loss : 9.949554 Content Loss: 22.987677

run 310:
Style Loss : 9.392783 Content Loss: 22.834240

run 320:
Style Loss : 8.980922 Content Loss: 22.719200

run 330:
Style Loss : 8.566155 Content Loss: 22.604099

run 340:
Style Loss : 8.203078 Content Loss: 22.458809

run 350:
Style Loss : 7.881061 Content Loss: 22.333389

run 360:
Style Loss : 7.596567 Content Loss: 22.200092

run 370:
Style Loss : 7.325222 Content Loss: 22.064774

run 380:
Style Loss : 7.067261 Content Loss: 21.956779

run 390:
Style Loss : 6.890582 Content Loss: 21.865513

run 400:
Style Loss : 6.699574 Content Loss: 21.745445

run 410:
Style Loss : 6.495186 Content Loss: 21.642336

run 420:
Style Loss : 6.334900 Content Loss